# GMS Authentication

## Background

As part of NGIS Genomics England are migrating from LDAP and adopting Active Directory (AD). This means that the authentication in the Interpretation API (aka CIP-API ) will also be moved to AD. 

This will impact the  way systems such as internal pipelines, interpretation services (external and internal) and decision support systems (DSS) authenticate against the CIP-API. 

The following notebook provides an example of how to use Python to obtain an AD Authentication token for system access to the CIP-API aka "Client Credentials Flow" 

### Authentication Method

The authentication method will be the same we are using at the moment, based on a `Bearer Token` present in the header of each request. 

This token will have an expiration time, clients should get a new one when this happens. 


In [ ]:
tenant_id = "afee026d-8f37-400e-8869-72d9124873e4" # this is the ID of the Active Dirctory "tenant"
client_id = "" # this is the ID of the client (e.g. a gel2mdt application) that is going to query the CIP-API
client_secret = "" # this is the password for the client that is going to query the CIP-API

_**If you are developing (or working with) an application (e.g. gel2mdt) that requires system access to the CIP-API please contact Genomics England Service Desk to obtain details of the `tenant_id`, `client_id` and `client_sceret`**_

In [ ]:
import requests
 
url = "https://login.microsoftonline.com/{tenant_id}/oauth2/token".format(tenant_id=tenant_id)

payload = "grant_type=client_credentials"

headers = {
    'Content-Type': "application/x-www-form-urlencoded",
    }

response = requests.request("POST", url, data=payload, headers=headers, auth=(client_id, client_secret))

print(response.text)

In [ ]:
token = response['access_token']

In [ ]:
token

## Example using AD authentication token with requests library

In [ ]:
interpretation_request_id = "88"
interpretation_request_version = "1"

In [ ]:
def get_ir(token, interpretation_request_id, interpretation_request_version, extra_params="?extra-params=show_referral"):
    
    """ Get the complete interpretation request data for a specific case and version 
    
    uses an extra parameter to get the referral information for GMS cases
    
    """
    
    url = '{host}/api/2/interpretation-request/{ir_id}/{ir_v}/{extras}'.format(host=host, 
                                                                               ir_id=interpretation_request_id, 
                                                                               ir_v=interpretation_request_version, 
                                                                               extras=extra_params)
    auth_header = {'Authorization': 'JWT {}'.format(token)}
    tv = requests.get(url, headers=auth_header)
    
    return tv

    # check that the ir has the interpretation request data
#     if 'interpretation_request_data' in tv and tv['interpretation_request_data']:
#         return tv
#     else:
#         raise Exception('Case not found')

In [ ]:
ir = get_ir(token=token, interpretation_request_id=interpretation_request_id, 
            interpretation_request_version=interpretation_request_version)

In [ ]:
ir

In [ ]:
ir.json()

## Example using AD authentication token with pycipapi client

In [ ]:
host = "https://cipapi-gms-beta.gel.zone"
from pycipapi.cipapi_client import CipApiClient
cipapi = CipApiClient(host, token=token) # this creates a cip-api client which can then be used to make additional queries of the CIP-API data

In [ ]:
for referral in cipapi.list_referral():
    print('Referral id: {}'.format(referral.referral_id))